In [ ]:
import pandas as pd

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
#give the input text to the variable gloss_text
gloss_text = "all feel started"
tokens = nltk.word_tokenize(gloss_text)
lemmatizer = WordNetLemmatizer()
gloss_vals = []
# Iterate over each token
for token in tokens:
  lemma = lemmatizer.lemmatize(token, pos='v')
  gloss_vals.append(lemma)

gloss_text = " ".join(gloss_vals)
print(gloss_text)

In [107]:
videos = pd.read_csv("/content/video-metadata.csv")

In [112]:
def get_pose_ids(gloss):
    gloss_list = gloss.split(" ")
    pose_ids = list()
    videos["Gloss Variant"] = videos["Gloss Variant"].apply(lambda x: x.strip("+#_()123456789"))
    for term in gloss_list:
        ids = videos[videos["Gloss Variant"] == term]["id"].values
        if len(ids) > 0:
            pose_ids.append(ids[0])
    return pose_ids

In [109]:
def get_pose_files(lookup_directory, pose_ids):
    pose_filepaths = []
    for pose_id in pose_ids:
        pose_filename = "pose-{}.mov".format(pose_id)
        pose_filepath = os.path.join(lookup_directory, pose_filename)
        pose_filepaths.append(pose_filepath)
    return pose_filepaths

In [ ]:
pose_ids = get_pose_ids(gloss_text.upper())
gloss_available = []
for i in pose_ids:
  gloss_check = videos[videos["id"] == i]["Gloss Variant"]
  gloss_available.append(gloss_check)

gloss_available

In [ ]:
#upload the dataset to google drive and mount it
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#untar the file
!tar -xvf "/content/drive/MyDrive/gloss2pose-lookup.tar.gz" -C "/content/"

In [118]:
import os

In [119]:
video_filenames = get_pose_files('/content/lookup/', pose_ids)

In [ ]:
# creating a .mp4 file from .mov
import os.path
from os import path
video_filenames_mp4 = []
for every_video in video_filenames:
  input_video = every_video
  output_video = '/content/lookup_gloss_videolist/lookup_mp4/'+input_video.split('/')[3].split('.')[0]+'.mp4'
  video_filenames_mp4.append('/content/lookup_gloss_videolist/lookup_mp4/'+input_video.split('/')[3].split('.')[0]+'.mp4')
  if path.exists(output_video) == False:
    print("convertion to mp4 is required for: ",input_video)
    !ffmpeg -i $input_video -b:v 1M -vcodec libx264 -preset slow -profile:v baseline -level 3.0 -pix_fmt yuv420p -movflags +faststart $output_video


In [127]:
# Creating the content to merge the required video files
updated_gloss = gloss_text.upper().replace(" ","_")
gloss_videolist = "/content/lookup_gloss_videolist/"+updated_gloss+".txt"
gloss_videolist_to_write = "file '/content/lookup_gloss_videolist/"+updated_gloss+".txt'"
if path.exists(gloss_videolist) == False:
   with open(gloss_videolist, "w") as f:
      # Write each value in the list as a new line in the file
      for item in video_filenames_mp4:
        f.write("file " + item + "\n")


In [ ]:
with open(gloss_videolist, "r") as f:
    file_content = f.read()
    print(file_content)

In [124]:
print(updated_gloss)

ALL_FEEL_START


In [ ]:
#Merging all the relavant videos
!ffmpeg -safe 0 -f concat -i "/content/lookup_gloss_videolist/ALL_FEEL_START.txt" -c copy "/content/lookup_gloss_videolist/ALL_FEEL_START.mp4"